In [2]:
# Continue with regular imports

import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import time
import os
import copy
import sklearn.svm
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix  
from collections import Counter
import random

import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchvision import datasets
from torch.utils.data import DataLoader

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the Pytorch_Modules directory, download it from GitHub if it doesn't work
try:
    from Pytorch_Modules import engine, prediction, utils, alexnet
except:
    # Get the Pytorch_Modules scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/yasinahmadpoor/FromScratch
    !mv FromScratch/Pytorch_Modules .
    !rm -rf FromScratch
    from Pytorch_Modules import engine, prediction, utils


device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
def get_train_test_loader(data_dir,
                           batch_size,
                           augment,
                           shuffle=True):

    # define normalizer
    normalize = transforms.Normalize(
            mean=[0.4914, 0.4822, 0.4465],
            std=[0.2023, 0.1994, 0.2010])    # Normalize a tensor image with mean and standard deviation for n channels, this transform will normalize each channel of the input.


    # define transforms
    valid_transform = transforms.Compose([
                transforms.Resize((227,227)),
                transforms.ToTensor(),          # transforms.ToTensor() automatically scales the images to [0,1] range
                normalize])


    if augment:
            train_transform = transforms.Compose([
                transforms.RandomCrop(32, padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                normalize,
            ])
    else:
            train_transform = transforms.Compose([
                transforms.Resize((227,227)),
                transforms.ToTensor(),
                normalize,
            ])

    # download and create datasets
    train_dataset = datasets.CIFAR10(root=data_dir, train=True, transform=train_transform, download=True)
    valid_dataset = datasets.CIFAR10(root=data_dir, train=False, transform=valid_transform, download=True)

    # define the data loaders
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle = shuffle)
    valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=shuffle)

    return (train_dataset, valid_dataset, train_loader, valid_loader)

BATCH_SIZE = 64
train_data, test_data, train_dataloader, test_dataloader = get_train_test_loader(data_dir = 'CIFAR10_data',
                                                    batch_size = BATCH_SIZE,
                                                    augment = False)
# Let's check out what we've created
print(f"Dataloaders: {train_dataloader, test_dataloader}") 
print(f"Length of train dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}")
print(f"Length of test dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}")

Files already downloaded and verified
Files already downloaded and verified
Dataloaders: (<torch.utils.data.dataloader.DataLoader object at 0x7fbf1678cd30>, <torch.utils.data.dataloader.DataLoader object at 0x7fbf1678cf10>)
Length of train dataloader: 782 batches of 64
Length of test dataloader: 157 batches of 64


In [6]:
next(iter(train_dataloader))[0].shape

torch.Size([64, 3, 227, 227])

In [3]:
def set_up_network(freeze_training = True, pretrained=True, clip_classifier = True):    
    network = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=pretrained)

    if clip_classifier:
        features = list(network.classifier.children())[:-1] # Remove last layer
        network.classifier = nn.Sequential(*features) # Replace the model classifier

    if freeze_training:
        for param in network.parameters():
            param.require_grad = False
    return network

In [4]:
models = set_up_network(freeze_training = True, pretrained=True, clip_classifier = True)

summary(model=models, 
        input_size=(32, 3, 227, 227), # make sure this is "input_size", not "input_shape"
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
AlexNet (AlexNet)                        [32, 3, 227, 227]    [32, 4096]           --                   True
├─Sequential (features)                  [32, 3, 227, 227]    [32, 256, 6, 6]      --                   True
│    └─Conv2d (0)                        [32, 3, 227, 227]    [32, 64, 56, 56]     23,296               True
│    └─ReLU (1)                          [32, 64, 56, 56]     [32, 64, 56, 56]     --                   --
│    └─MaxPool2d (2)                     [32, 64, 56, 56]     [32, 64, 27, 27]     --                   --
│    └─Conv2d (3)                        [32, 64, 27, 27]     [32, 192, 27, 27]    307,392              True
│    └─ReLU (4)                          [32, 192, 27, 27]    [32, 192, 27, 27]    --                   --
│    └─MaxPool2d (5)                     [32, 192, 27, 27]    [32, 192, 13, 13]    --                   --
│    └─Conv2d (6)     

In [7]:

def set_AlexNet():
    # Set the manual seeds
    torch.manual_seed(42)
    torch.cuda.manual_seed(42)

    model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
    # model.eval()


    # Freeze all base layers in the "features" section of the model (the feature extractor) by setting requires_grad=False
    for param in model.parameters():
      param.requires_grad = False

    features = list(model.classifier.children())[:-1] # Remove last layer
    model.classifier = nn.Sequential(*features) # Replace the model classifier
    model.classifier[4].requires_grad = True

    return model


In [8]:
# Print a summary using torchinfo (uncomment for actual output)
model = set_AlexNet()
summary(model=model, 
        input_size=(32, 3, 227, 227), # make sure this is "input_size", not "input_shape"
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
AlexNet (AlexNet)                        [32, 3, 227, 227]    [32, 4096]           --                   False
├─Sequential (features)                  [32, 3, 227, 227]    [32, 256, 6, 6]      --                   False
│    └─Conv2d (0)                        [32, 3, 227, 227]    [32, 64, 56, 56]     (23,296)             False
│    └─ReLU (1)                          [32, 64, 56, 56]     [32, 64, 56, 56]     --                   --
│    └─MaxPool2d (2)                     [32, 64, 56, 56]     [32, 64, 27, 27]     --                   --
│    └─Conv2d (3)                        [32, 64, 27, 27]     [32, 192, 27, 27]    (307,392)            False
│    └─ReLU (4)                          [32, 192, 27, 27]    [32, 192, 27, 27]    --                   --
│    └─MaxPool2d (5)                     [32, 192, 27, 27]    [32, 192, 13, 13]    --                   --
│    └─Conv2d (6) 

In [10]:
# Extract features from the training data
train_features = []
train_labels = []
model = set_AlexNet()
model.eval()

for X,y in train_dataloader:
  features = model(X)
  train_features.append(features)
  train_labels.append(y)




print(train_features.shape)
print(train_labels)


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


KeyboardInterrupt: ignored

In [ ]:
import numpy as np
import os
from keras.preprocessing import image
from keras.applications import AlexNet
from keras.models import Model
from sklearn.svm import SVC

# Load the pre-trained AlexNet model
alexnet = AlexNet(weights='imagenet')

# Remove the last layer (the output layer) from the model
alexnet = Model(inputs=alexnet.inputs, outputs=alexnet.layers[-2].output)

# Set up the paths to your training and test data directories
train_dir = 'path/to/your/training/data/directory'
test_dir = 'path/to/your/test/data/directory'

# Set up the parameters for feature extraction and SVM classification
num_classes = 10  # number of classes in your dataset
batch_size = 64  # batch size for feature extraction
C = 1.0  # SVM regularization parameter

# Extract features from the training data
train_features = []
train_labels = []

for subdir in os.listdir(train_dir):
    label = int(subdir)  # the subdirectory name is the class label
    subpath = os.path.join(train_dir, subdir)
    for file in os.listdir(subpath):
        img_path = os.path.join(subpath, file)
        img = image.load_img(img_path, target_size=(227, 227))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        features = alexnet.predict(x)
        train_features.append(features)
        train_labels.append(label)

train_features = np.array(train_features).reshape(-1, 4096)
train_labels = np.array(train_labels)

# Train an SVM model
svm = SVC(kernel='linear', C=C)
svm.fit(train_features, train_labels)

# Extract features from the test data and use the SVM model to predict the labels
test_features = []
test_filenames = []

for file in os.listdir(test_dir):
    img_path = os.path.join(test_dir, file)
    img = image.load_img(img_path, target_size=(227, 227))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = alexnet.predict(x)
    test_features.append(features)
    test_filenames.append(file)

test_features = np.array(test_features).reshape(-1, 4096)
test_labels = svm.predict(test_features)

# Print the predicted labels for the test data
for i in range(len(test_filenames)):
    print(test_filenames[i], test_labels[i])